In [2]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [1]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 778kB 8.8MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 3.0MB 12.8MB/s 
     |████████████████████████████████| 317kB 31.1MB/s 
     |████████████████████████████████| 1.1MB 31.2MB/s 
     |████████████████████████████████| 890kB 40.4MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=15128c5a62a110e86db817c6fe91bc86207ebac5d87ddc1f3d556b0c4e006b15
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7a6129235fba893f1e91080d32e5c98b59538b0c750b0d539ccc768417e0f4c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses
ERROR: transformers 3.0.2 has requirement toke

In [14]:
!pip install apex

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 716kB 8.3MB/s 
     |████████████████████████████████| 327kB 18.0MB/s 
     |████████████████████████████████| 174kB 19.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 235kB 22.4MB/s 
     |████████████████████████████████| 143kB 19.4MB/s 
     |████████████████████████████████| 122kB 20.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for cryptacular: filename=cryptacular-1.5.5-cp36-abi3-manylinux2010_x86_64.whl size=47561 sha256=c2fdfd5f67f70eee40a21154e9c8576777e993213fefc48d1b5802a2160904a6
  Stored in directory: /root/.cache/pip/wheels/3c/79/bc/1eec7120c3ff9b0a2c7ad94d1626abc3388688e2ed7a45878f
Successfully built cryptacular
  Created wheel for apex: filename=apex-0.9.10.dev0-cp36-none-any.whl size=46467 sha256=f533d65ff1d97d4519d77a162fa

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.model_selection import train_test_split

In [7]:
speech = pd.read_table("search_dataset_small.txt", sep="|")

sp = speech['speech']
party = speech['party'].apply(lambda x: 1 if x == 'R' else 0)

df = pd.DataFrame({
    'text': sp,
    'labels': party
})

train_df, eval_df = train_test_split(df, test_size=0.2)

In [9]:
train_df.head(15)

,text,labels
21,"Mr. President, as the Congress and the America...",1
0,"Mr. President, I wish to report on behalf of m...",1
56,"Mr. Speaker, I thank the Chair very much for t...",1
42,"Mr. Speaker, I rise in opposition to the rule ...",1
2,"Madam President, I send to the desk a resoluti...",1
90,"Mr. Speaker, I thank the gentleman from Colora...",1
84,"Mr. Speaker, I commend my distinguished collea...",0
58,four other members of the Long Island delegati...,1
62,Thank you. Mr. Olson. Let me just ask you this...,0
43,"Mr. Speaker, I find it ironic that a party whi...",1


In [10]:
print(len(train_df), len(eval_df))

80 20


In [11]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [18]:
model_args = ClassificationArgs(num_train_epochs=1, fp16=False)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)
model.train_model(train_df)

- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [19]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 8, 'tn': 0, 'fp': 12, 'fn': 0, 'eval_loss': 0.7509103417396545}


In [20]:
result

{'eval_loss': 0.7509103417396545,
 'fn': 0,
 'fp': 12,
 'mcc': 0.0,
 'tn': 0,
 'tp': 8}